In [1]:
# Relevant Python Libraries  

import pandas as pd                         # for data manupulation                   
import matplotlib.pyplot as plt             # for data visualization
from sklearn.impute import SimpleImputer    # for data impution 
import numpy as np                          # for numeracal computation
from linearmodels.panel import PanelOLS     # for statistical modeling 

In [3]:
# import data as dataframe for analysis 

df = pd.read_excel('/Users/davidsokurov/Desktop/I.S./Data Analysis/Final I.S. Dataset.xlsx')

The data is processed and analyzed in the following 4 steps:

1. Data cleaning, wrangling and organization 
2. Exploratory Data Analysis (EDA)
3. Regression Modeling and Analysis
4. Regression Diagnostic checks 

#### Step 1. Data cleaning, wrangling and organization 

In [ ]:
# Imputes the variables Visitation and Tourism
df['Visitation'] = df.groupby('Park')['Visitation'].transform(lambda x: x.fillna(x.mean()))
df['Tourism'] = df.groupby('Park')['Tourism'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# Total missing values are 51 (Visitation and Tourism)
missing_values = df.isnull().sum()
print(missing_values)
total_missing_values = df.isnull().sum().sum()

print("Total missing values:", total_missing_values)

In [ ]:
# Get the list of variables 
variable_list = df.columns.tolist()
print("List of variables:", variable_list)

In [ ]:
unique_parks = df['Park'].unique()
unique_park_count = df['Park'].nunique()
print(unique_park_count)
print(unique_parks)

In [ ]:
# GIves the list of all missing values and their locations 
missing_values_df = df.isnull()

missing_locations = pd.DataFrame([(row, col) for row in missing_values_df.index for col in missing_values_df.columns if missing_values_df.at[row, col]])
print("Locations of missing values:")
print(missing_locations)

In [ ]:
# Saves the new dataset as a csv file
df = df.to_csv("final_data.csv", index=False)
#df.to_excel("edited_dataset.xlsx")

In [ ]:
unique_countries = df["Country"].unique()
print(unique_countries)

#### Step 2. Exploratory Data Analysis 

In [ ]:
for park, data in df.groupby('Park'):
    plt.plot(data['Year'], data['Visitation'], label=park)

plt.title('Visitation Trends Over the Years')
plt.xlabel('Year')
plt.ylabel('Visitation')

# Create a separate legend object
legend = plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

plt.figure(figsize=(15, 6))  

plt.show()

In [ ]:
# add numbers on bargraphs 
mean_visitation_by_country = df.groupby('Country')['Visitation'].mean()

# Sort countries in ascending order based on mean visitation
sorted_countries = mean_visitation_by_country.sort_values().index

plt.figure(figsize=(20, 10))
for country in sorted_countries:
    plt.bar(country, mean_visitation_by_country[country], label=country)
    plt.text(country, mean_visitation_by_country[country], f'{mean_visitation_by_country[country]:.2f}', ha='center', va='bottom')

plt.title('Visitation Across Countries (Ascending Order)')
plt.xlabel('Country')
plt.ylabel('Mean Visitation')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
df.groupby('Year')['Visitation'].mean().plot(marker='o')
plt.title('Mean Visitation Over the Years')
plt.xlabel('Year')
plt.ylabel('Mean Visitation')
plt.show()

In [ ]:
# MAKE IT HORIZONTAL
plt.figure(figsize=(15, 8))
tourism_by_country = df.groupby('Country')['Tourism'].mean().sort_values(ascending=True)
tourism_by_country.plot(kind='barh', color='skyblue')

for index, value in enumerate(tourism_by_country):
    plt.text(value, index, f'{value:,}', va='center', fontsize=10)

plt.title('Total Tourism by Country')
plt.xlabel('Total Tourism')
plt.ylabel('Country')
plt.show()

In [ ]:
# Get the list of unique countries
countries = df['Country'].unique()

# Create subplots for each country
fig, axes = plt.subplots(nrows=len(countries), ncols=1, figsize=(10, 3 * len(countries)))

# Iterate over countries and create scatter plots with trend lines
for i, country in enumerate(countries):
    country_data = df[df['Country'] == country]
    
    # Scatter Plot: Animation Popularity vs. Visitation
    axes[i].scatter(x='AnimationPopularity', y='Visitation', data=country_data, alpha=0.5, label='Visitation')
    
    # Trend line for Animation Popularity vs. Visitation
    z_visitation = np.polyfit(country_data['AnimationPopularity'], country_data['Visitation'], 1)
    p_visitation = np.poly1d(z_visitation)
    axes[i].plot(country_data['AnimationPopularity'], p_visitation(country_data['AnimationPopularity']), color='red')

    # Scatter Plot: Animation Popularity vs. Tourism
    axes[i].scatter(x='AnimationPopularity', y='Tourism', data=country_data, alpha=0.5, label='Tourism')
    
    # Trend line for Animation Popularity vs. Tourism
    z_tourism = np.polyfit(country_data['AnimationPopularity'], country_data['Tourism'], 1)
    p_tourism = np.poly1d(z_tourism)
    axes[i].plot(country_data['AnimationPopularity'], p_tourism(country_data['AnimationPopularity']), color='blue')

    axes[i].set_title(f'Scatter Plot: Animation Popularity, Visitation, and Tourism - {country}')
    axes[i].set_xlabel('Animation Popularity')
    axes[i].set_ylabel('Count')
    axes[i].legend()

plt.tight_layout()
plt.show()


#### Step 3. Regression

In [ ]:
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Set the DataFrame index using the panel variables and sort it
df.set_index(['Park', 'Year'], inplace=True)
df.sort_index(inplace=True)

# Create a PanelOLS model
model = PanelOLS.from_formula('Visitation ~ AnimationPopularity + Tourism + GDP + Population', data=df)

# Fit the model
result = model.fit()

# Print regression results
print(result)

#### Step 4. Diagnostics Checks and Adjusting the Model 

In [ ]:
df = pd.read_excel('/Users/davidsokurov/Desktop/I.S./Data Analysis/Final I.S. Dataset.xlsx')
df.set_index(["Park", "Year"], inplace=True)


df['Visitation'] = df.groupby('Park')['Visitation'].transform(lambda x: x.fillna(x.mean()))
df['Tourism'] = df.groupby('Park')['Tourism'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
years = df.index.get_level_values("Year").to_list()
df["Year"] = pd.Categorical(years)

In [ ]:
correlation_matrix = df[["AnimationPopularity", "Tourism", "GDP", "Population"]].corr()
print(correlation_matrix)


In [ ]:
# Perform PooledOLS
from linearmodels import PooledOLS
import statsmodels.api as sm
exog = sm.tools.tools.add_constant(df["AnimationPopularity"])

endog = df["Visitation"]
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

In [ ]:
# 3A. Homoskedasticity
import matplotlib.pyplot as plt
 # 3A.1 Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = "blue")
ax.axhline(0, color = 'r', ls = '--')
ax.set_xlabel("Predicted Values", fontsize = 15)
ax.set_ylabel("Residuals", fontsize = 15)
ax.set_title("Homoskedasticity Test", fontsize = 12)
plt.show()

In [ ]:
# 3A.2 White-Test
from statsmodels.stats.diagnostic import het_white, het_breuschpagan

# Assuming you have residuals_pooled_OLS from your previous code
pooled_OLS_dataset = pd.concat([df, residuals_pooled_OLS.rename("residuals")], axis=1)
pooled_OLS_dataset = pooled_OLS_dataset.drop(["Year"], axis=1).fillna(0)
exog = sm.tools.tools.add_constant(df["AnimationPopularity"]).fillna(0)
white_test_results = het_white(pooled_OLS_dataset["residuals"], exog)
labels = ["LM-Stat", "LM p-val", "F-Stat", "F p-val"] 
print(dict(zip(labels, white_test_results)))

# 3A.3 Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset["residuals"], exog)
labels = ["LM-Stat", "LM p-val", "F-Stat", "F p-val"] 
print(dict(zip(labels, breusch_pagan_test_results)))


In summary, the statistical tests indicate that the assumption of homoskedasticity is likely violated, and there is evidence to support the presence of heteroskedasticity in the residuals.

In [ ]:
# 3.B Non-Autocorrelation
# Durbin-Watson-Test
from statsmodels.stats.stattools import durbin_watson

durbin_watson_test_results = durbin_watson(pooled_OLS_dataset["residuals"]) 
print(durbin_watson_test_results)

Durbin-Watson statistic of 0.41 is significantly less than 2, indicating positive autocorrelation in the residuals. This suggests that there may be a pattern in the residuals that is not explained by the model, and there might be some temporal dependence between consecutive observations.

As a consequence, assumption 3b is also violated, so it seems that a FE-/RE-model will be more suitable.

In [ ]:
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.api as sm

# Assuming df contains 'AnimationPopularity', 'Tourism', 'GDP', 'Population', and 'Visitation'
exog_vars = ['AnimationPopularity', 'Tourism', 'Population']
exog = sm.tools.tools.add_constant(df[exog_vars])
endog = df['Visitation']

# Random effects model
model_re = RandomEffects(endog, exog, check_rank=False) 
re_res = model_re.fit() 

# Fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects=True) 
fe_res = model_fe.fit() 

# Print results
print(re_res)


In [ ]:
print(fe_res)

In [ ]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad
from scipy import stats

# Log transformation of the dependent variable
df['Log_Visitation'] = np.log(df['Visitation'])

# Define independent and dependent variables
X = df[['AnimationPopularity', 'Tourism', 'GDPPerCapita',]]
y = df['Log_Visitation']

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())

# Test for multicollinearity
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

print("\nVariance Inflation Factor (VIF):")
print(calculate_vif(X))

# Test for homoskedasticity
_, p_homoskedasticity, _, _ = het_breuschpagan(model.resid, X)
print("\nHomoskedasticity Test (Breusch-Pagan):")
print("p-value:", p_homoskedasticity)

# Test for normality of residuals
p_normality = stats.normaltest(model.resid)[1]
print("\nNormality Test (Jarque-Bera):")
print("p-value:", p_normality)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad
from scipy import stats

# Log transformation of the dependent variable
df['Log_Visitation'] = np.log(df['Visitation'])

# Log transformation of independent variables
df['Log_Tourism'] = np.log(df['Tourism'])
df['Log_GDPPerCapita'] = np.log(df['GDPPerCapita'])

# Define independent and dependent variables
X = df[['AnimationPopularity', 'Log_Tourism', 'Log_GDPPerCapita']]
y = df['Log_Visitation']

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())

# Test for multicollinearity
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

print("\nVariance Inflation Factor (VIF):")
print(calculate_vif(X))

# Test for homoskedasticity
_, p_homoskedasticity, _, _ = het_breuschpagan(model.resid, X)
print("\nHomoskedasticity Test (Breusch-Pagan):")
print("p-value:", p_homoskedasticity)

# Test for normality of residuals
p_normality = stats.normaltest(model.resid)[1]
print("\nNormality Test (Jarque-Bera):")
print("p-value:", p_normality)